In [36]:
import requests as req
import pandas as pd

In [57]:
# fetch movies names descriptions and genres from given api

movies_df = pd.DataFrame()

url = "https://api.themoviedb.org/3/movie/top_rated?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US&page="
total_pages = 100
movies_data = []

for p in range(1, total_pages+1):
    final_url = url + str(p)
    res = req.get(final_url)
    if(res.status_code==200):
        data = res.json()
        movies_data.extend(data['results'])

print('complete!')

complete!


In [58]:
# fetch genres against their id from the given api

genres_dict = {}

url = "https://api.themoviedb.org/3/genre/movie/list?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US"

res = req.get(url)
if(res.status_code == 200):
    data = res.json()
    genres_dict = {genre['id']: genre['name'] for genre in data['genres']}
    print(genres_dict)

print('complete!')

{28: 'Action', 12: 'Adventure', 16: 'Animation', 35: 'Comedy', 80: 'Crime', 99: 'Documentary', 18: 'Drama', 10751: 'Family', 14: 'Fantasy', 36: 'History', 27: 'Horror', 10402: 'Music', 9648: 'Mystery', 10749: 'Romance', 878: 'Science Fiction', 10770: 'TV Movie', 53: 'Thriller', 10752: 'War', 37: 'Western'}
complete!


In [59]:
# convert json data into pandas datafram ad show first 5 rows

movies_df = pd.DataFrame(movies_data)
movies_df = movies_df[['title', 'genre_ids', 'overview']]
movies_df.head()

,title,genre_ids,overview
0,The Shawshank Redemption,"[18, 80]",Imprisoned in the 1940s for the double murder ...
1,The Godfather,"[18, 80]","Spanning the years 1945 to 1955, a chronicle o..."
2,The Godfather Part II,"[18, 80]",In the continuing saga of the Corleone crime f...
3,Schindler's List,"[18, 36, 10752]",The true story of how businessman Oskar Schind...
4,12 Angry Men,[18],The defense and the prosecution have rested an...


In [63]:
# add new column of genre names by matching genre_ids from the fetched genres dictionary

movies_df['genres'] = movies_df['genre_ids'].apply(lambda ids: [genres_dict[i] for i in ids])
movies_df = movies_df.drop(columns = ['genre_ids'])
movies_df.head()

,title,overview,genres
0,The Shawshank Redemption,Imprisoned in the 1940s for the double murder ...,"[Drama, Crime]"
1,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","[Drama, Crime]"
2,The Godfather Part II,In the continuing saga of the Corleone crime f...,"[Drama, Crime]"
3,Schindler's List,The true story of how businessman Oskar Schind...,"[Drama, History, War]"
4,12 Angry Men,The defense and the prosecution have rested an...,[Drama]
